## Make labels

In [63]:
from skimage.io import imread, imsave
import napari
import os
import numpy as np
from pathlib import Path
from tnia.deeplearning.dl_helper import make_label_directory

tnia_images_path = Path(r'D:/images')
parent_path=Path(tnia_images_path / r'tnia-python-images\imagesc\2024_07_03_dl_scale_shape')
label_path = Path(parent_path / r'labels')

name = 'phantom2'
ext = '.tif'

num_classes = 1

image_label_paths, mask_label_paths = make_label_directory(1, num_classes, label_path)

for mask_label_name in image_label_paths:
    print(mask_label_name)
print()

im = imread(os.path.join(parent_path / (name+ext)))

labels = []

try:
    for mask_label_path in mask_label_paths:

        for i in range(num_classes):
            label = imread(os.path.join(mask_label_path / (name+".tif")))
            labels.append(label)
except:
    print('labels not created yet', (os.path.join(label_path / (name+".tif"))))
    for i in range(num_classes):
        labels.append(np.zeros([im.shape[0], im.shape[1]], dtype=np.uint8))

print('there are', len(labels), 'labels')


D:\images\tnia-python-images\imagesc\2024_07_03_dl_scale_shape\labels\input0

labels not created yet D:\images\tnia-python-images\imagesc\2024_07_03_dl_scale_shape\labels\phantom2.tif
there are 1 labels


## Edit an ROI of the labels in Napari

Now edit the labels in Napari.  We can start from a blank image or alternatively a pre-existing set of labels (either generated by another method, or previous manual labels) that we want to improve.

Use the box ROI to indicate which region of the image contains labels.

In [64]:
viewer = napari.Viewer()
viewer.add_image(im, name='im')
for i in range(num_classes):
    viewer.add_labels(labels[i], name='labels'+str(i))

boxes_layer = viewer.add_shapes(
            name="Label box",
            face_color="transparent",
            edge_color="green",
            edge_width=2,
        )

boxes_layer.add([[[0,0],[im.shape[0],im.shape[1]//2]]])

## Get the ROI that was labeled

In [65]:
ystart = int(np.min(boxes_layer.data[0][:,0]))
yend = int(np.max(boxes_layer.data[0][:,0]))
xstart = int(np.min(boxes_layer.data[0][:,1]))
xend = int(np.max(boxes_layer.data[0][:,1]))

print('bounding box is',ystart, yend, xstart, xend)
if np.ndim(im) == 3:
    im_cropped = im[ystart:yend, xstart:xend, :]
else:
    im_cropped = im[ystart:yend, xstart:xend]

labels_cropped = []
for i in range(num_classes):
    temp = labels[i][ystart:yend, xstart:xend]
    labels_cropped.append(temp)

print(im_cropped.shape, labels_cropped[0].shape)

bounding box is 642 990 37 549
(348, 512) (348, 512)



## Save modified labels

In [66]:
from tnia.deeplearning.dl_helper import generate_patch_names, generate_next_patch_name

image_name, mask_name = generate_patch_names(str(image_label_paths[0]), str(mask_label_path), name)

base_name = generate_next_patch_name(str(image_label_paths[0]), name)

print(base_name)

print(image_name)
print(mask_name)
print(base_name)


phantom2_1
D:\images\tnia-python-images\imagesc\2024_07_03_dl_scale_shape\labels\input0/phantom2_1.tif
D:\images\tnia-python-images\imagesc\2024_07_03_dl_scale_shape\labels\ground truth0/phantom2_1.tif
phantom2_1


In [67]:
imsave(image_name, im_cropped)
print(image_name)

for i in range(num_classes):
    print(mask_label_paths[i])
    imsave(os.path.join(mask_label_paths[i], base_name+'.tif'), labels_cropped[i])

D:\images\tnia-python-images\imagesc\2024_07_03_dl_scale_shape\labels\input0/phantom2_1.tif
D:\images\tnia-python-images\imagesc\2024_07_03_dl_scale_shape\labels\ground truth0


C:\Users\bnort\AppData\Local\Temp\ipykernel_35596\2409316094.py:6: UserWarning: D:\images\tnia-python-images\imagesc\2024_07_03_dl_scale_shape\labels\ground truth0\phantom2_1.tif is a low contrast image
  imsave(os.path.join(mask_label_paths[i], base_name+'.tif'), labels_cropped[i])


In [68]:
print(image_name)
json_name = os.path.basename(image_name).split('.')[0]+'.json'
print(json_name)

D:\images\tnia-python-images\imagesc\2024_07_03_dl_scale_shape\labels\input0/phantom2_1.tif
phantom2_1.json


In [69]:
# take away extension from image_name and replace with json
json_name = os.path.basename(image_name).split('.')[0]+'.json'

# save xstart, ystart, xend, yend to json 
import json
json_name = os.path.join(image_label_paths[0], json_name)
with open(json_name, 'w') as f:
    json_ = {}
    json_['source_name'] = name + ext
    json_['bbox'] = [xstart, ystart, xend, yend]
    json.dump(json_, f)